In [1]:
#importing required libraries
import os  # for input output
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from collections import Counter
    
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint



In [2]:
#setting the working directory

os.chdir("C:/Users/dell/PYTHON DATA SCIENCE WORK")
print(os.getcwd())

C:\Users\dell\PYTHON DATA SCIENCE WORK


In [ ]:
#loading the data
train=pd.read_csv("train_cab.csv",na_values = {"pickup_datetime":"43"})
test =pd.read_csv("test_cab.csv")

In [ ]:
train.head()

# Understanding the dataset

In [ ]:
test.head() # fare_amt is not here

In [ ]:
train.shape

In [ ]:
train.dtypes  # OBJ TO NUMERIC ALWAYS

In [ ]:
test.dtypes

In [ ]:
train.describe()

In [ ]:
test.describe()

# Hyper parameter turing to check the parameter on which are model run fast
# Random search cv
# Grid search cv
# Haversine formula

# Data Pre Processing
# 1)data exploration and missing value analysis

In [ ]:
#converting fare amt from obj to numeric
train['fare_amount'] = pd.to_numeric(train['fare_amount'],errors='coerce') # this error will convert all the non numeric value to nan

In [ ]:
train.dtypes

In [ ]:
train.shape

In [ ]:
train.dropna(subset = ["pickup_datetime"])  # Dropping Na values in datetime col

# FOR TRAIN SET

In [ ]:
#pickup date_time is in obj we need to change its data type to datetime
train["pickup_datetime"] = pd.to_datetime(train["pickup_datetime"],format='%Y-%m-%d %H:%M:%S UTC')

In [ ]:
#seprate the pickup_datetime to seprate fields like year ,month and day
train['year']= train['pickup_datetime'].dt.year
train['Month']=train['pickup_datetime'].dt.month
train['Date']=train['pickup_datetime'].dt.day
train['Day']=train['pickup_datetime'].dt.dayofweek
train['Hour']=train['pickup_datetime'].dt.hour
train['Minute']=train['pickup_datetime'].dt.minute


In [ ]:
train.dtypes

# FOR TEST SET

In [ ]:
#pickup date_time is in obj we need to change its data type to datetime
test["pickup_datetime"] = pd.to_datetime(test["pickup_datetime"],format='%Y-%m-%d %H:%M:%S UTC')

In [ ]:
#seprate the pickup_datetime to seprate fields like year ,month and day
test['year']= test['pickup_datetime'].dt.year
test['Month']=test['pickup_datetime'].dt.month
test['Date']=test['pickup_datetime'].dt.day
test['Day']=test['pickup_datetime'].dt.dayofweek
test['Hour']=test['pickup_datetime'].dt.hour
test['Minute']=test['pickup_datetime'].dt.minute

In [ ]:
test.dtypes

# Missing value analysis

In [ ]:
# removing datetime missing value row
train = train.drop(train[train['pickup_datetime'].isnull()].index,axis=0)
print(train.shape)
print(train['pickup_datetime'].isnull().sum())

In [ ]:
train['passenger_count'].describe()

In [ ]:
#max number of passenger_count is 5345 which is actually not possible,so reducing passenger count to 6
train = train.drop(train[train['passenger_count']>6].index,axis=0)


In [ ]:
# also removing the value with passenger count =0
train = train.drop(train[train['passenger_count']==0].index,axis=0)


In [ ]:
train['passenger_count'].describe()

In [ ]:
train['passenger_count'].sort_values(ascending=True)

In [ ]:
#Removing passenger count missiong value row
train = train.drop(train[train['passenger_count'].isnull()].index,axis=0)
print(train.shape)
print(train['passenger_count'].isnull().sum())

In [ ]:
# THERE IS ONE PASSENGER COUNT OF 0.12 WHICH IS NOT POSSIBLE,HENCE REMOVE FRACTIONAL VALUE
train = train.drop(train[train['passenger_count']==0.12].index,axis=0)
train.shape


In [ ]:
# Finding descending order of fare to get to know hwter outlier present or not
train["fare_amount"].sort_values(ascending=False)

In [ ]:
#count the number of values in which fair amt is less then 0
Counter(train['fare_amount']<0)

In [ ]:
# remove the negative value in fare amt col
train = train.drop(train[train['fare_amount']<0].index,axis=0)
train.shape

In [ ]:
train['fare_amount'].min()

In [ ]:
# REMOVE THE ROW WHERE FAIR AMT IS 0
train = train.drop(train[train['fare_amount']<1].index,axis=0)
train.shape

In [ ]:
# There is a huge difference bw 1,2,3 pos in descending order of fair amt so we will remove the rows having fair amt 
train = train.drop(train[train['fare_amount']>454].index,axis=0)
train.shape

In [ ]:
train['fare_amount'].min()

In [ ]:
#Removing fair amt missiong value row
train = train.drop(train[train['fare_amount'].isnull()].index,axis=0)
print(train.shape)
print(train['fare_amount'].isnull().sum())

In [ ]:
train["fare_amount"].sort_values(ascending=False)

In [ ]:
train["fare_amount"].describe()

# Latitude can be from -90to90

# longitude can be from -180to108

In [ ]:
train[train['pickup_latitude']<-90]

In [ ]:
train[train['pickup_latitude']>90]

In [ ]:
# Removing the out of range value for latitude
train = train.drop(train[train['pickup_latitude']<-90].index,axis=0)
train = train.drop(train[train['pickup_latitude']>90].index,axis=0)
train.shape

In [ ]:
# Now checking the lngitude values
train[train['pickup_longitude']<-180]

In [ ]:
train[train['pickup_longitude']>180]

In [ ]:
# Removing the out of range value for latitude
train = train.drop(train[train['pickup_longitude']<-180].index,axis=0)
train = train.drop(train[train['pickup_longitude']>180].index,axis=0)
train.shape

In [ ]:
# Now checking for dropof longitude and latitude
train[train['dropoff_latitude']<-90]

In [ ]:
train[train['dropoff_latitude']>90]

In [ ]:
train[train['dropoff_longitude']<-180]

In [ ]:
train[train['dropoff_longitude']>180]

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

# CALCULATE DISTANCE BASED ON GIVEN ODINATES

In [ ]:
# IN THIS DATA SET WE HAVE BEEN GIVEN PICKUP LATTITUDES AND LONGITUDES AND DROP OFF LONGITUDE AND LATTITUDE
# WE NEED TO CALCUATE THE DISTANCE USING HAVERSINE FORMULA AND WE WILL CREATE A NEW VARIABLE CALLED DISTANCE
from math import radians ,cos , sin ,asin ,sqrt

def haversine(a):
    lon1=a[0]
    lat1=a[1]
    lon2=a[2]
    lat2=a[3]
    
    "calculating the great circle distance bw two points on earth"
    
    #convert decimal degree to radian
    lon1,lat1,lon2,lat2 = map(radians , [lon1,lat1,lon2,lat2])
    #haversin formula
    dlon=lon2-lon1
    dlat=lat2-lat1
    a= sin(dlat/2)**2 + cos(lat1)*cos(lat2)* sin(dlon/2)**2
    c= 2*asin(sqrt(a))
    #radius of earth in km is 6371
    km = 6371 * c
    return km

In [ ]:
train['distance'] = train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_longitude']].apply(haversine,axis=1)

In [ ]:
train.head()

In [ ]:
test['distance'] = test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_longitude']].apply(haversine,axis=1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.nunique()

In [ ]:
#finding descending order of distance to know if outlier is present or not?
train['distance'].sort_values(ascending = False)

In [ ]:
train['distance'].describe()

In [ ]:
train['distance'].max()

In [ ]:
train['distance'].min()

In [ ]:
train['distance'].value_counts()

THE TOP 23 DISTANCE VALUES IN THE DISTANCE VARIABLES ARE VERY HIGH , IT MEANS MORE THEN 8000 KMS DISTANCE THEY HAVE TRAVALLED
AFTER THIS IT GOES DOWN TO 127 THERE ARE OUTLIER

In [ ]:
train[train['distance']>8000]

In [ ]:
import matplotlib.pyplot as plt

# Plot a histogram
plt.hist(train['distance'], bins=50)
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Distance')
plt.show()

# Plot a box plot
plt.boxplot(train['distance'])
plt.ylabel('Distance')
plt.title('Box Plot of Distance')
plt.show()


In [ ]:
mean_distance = train['distance'].mean()
median_distance = train['distance'].median()
std_distance = train['distance'].std()

print(f"Mean Distance: {mean_distance}")
print(f"Median Distance: {median_distance}")
print(f"Standard Deviation: {std_distance}")


In [ ]:
threshold = mean_distance + 3 * std_distance


In [ ]:
threshold

In [ ]:
# we will remove the row distance value which is more then 129
train = train.drop(train[train['distance']>17732.50].index,axis=0)
train.shape

In [ ]:
#Counter(train['distance']==0)

In [ ]:
#Counter(test['distance']==0)

In [ ]:
#Counter(train['fare_amount']==0)

In [ ]:
# we will remove the row distance value which is 0
train = train.drop(train[train['distance']==0].index,axis=0)
train.shape

In [ ]:
test = test.drop(test[test['distance']==0].index,axis=0)
test.shape

In [ ]:
train.head()

 WE HAVE SPLITTED THE PICKUP_DATE TIME INTO YEAR,MONTH,DAY,HOUR,MINUTE SO WE ARE DROPPING PICKUP_DATE , dropping long and lat

In [ ]:
drop_columns = ['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','Minute']
train = train.drop(drop_columns,axis=1)

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
drop_columns = ['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','Minute']
test = test.drop(drop_columns,axis=1)

In [ ]:
test.dtypes

# Converting variables to Float Data Type to Int Data Type

In [ ]:
train['passenger_count'] = train['passenger_count'].astype('int64')
train['year'] = train['year'].astype('int64')
train['Month'] = train['Month'].astype('int64')
train['Date'] = train['Date'].astype('int64')
train['Day'] = train['Day'].astype('int64')
train['Hour'] = train['Hour'].astype('int64')

In [ ]:
train.dtypes

In [ ]:
train.head()

# Data Visualization
number of passanger effect the fair
pickup date and time effect the fair
day of the week does affect the fair
distance effect the fair

In [ ]:
#count plot
plt.figure(figsize=(15,7))
sns.countplot(x="passenger_count",data=train)

single and double passanger are frequent riders

In [ ]:
# Relationship bw number f passager and fair
plt.figure(figsize=(15,7))
plt.scatter(x=train['passenger_count'],y=train['fare_amount'],s=10)
plt.xlabel("Number of passangers")
plt.ylabel("Fare")

single and double passenger had the maximum fair

In [ ]:
# relationship bw date and fair
plt.figure(figsize=(15,7))
plt.scatter(x=train['Date'],y=train['fare_amount'],s=10)
plt.xlabel("Date")
plt.ylabel("Fare")

fair is almost equal in all days

In [ ]:
# relationshio bw the hour of day and fair
plt.figure(figsize=(15,7))
train.groupby(train['Hour'])['Hour'].count().plot(kind='bar')
plt.show()

lowest number of cabs were around 5am and highest number of cabs were around bw 6pm to 7 pm

In [ ]:
# relationship bw time and fare
plt.figure(figsize=(15,7))
plt.scatter(x=train['Hour'],y=train['fare_amount'],s=10)
plt.xlabel("Hour")
plt.ylabel("Fare")
plt.show()

Fare is highest at 7am and around 11 pm which tells the fair is highest in early  morning and late nights

In [ ]:
# relationship bw day of the week  and number of cab rides
plt.figure(figsize=(15,7))
sns.countplot(x='Day',data= train)

Not much relation bw day of the week and the count of number of cars

In [ ]:
# relationship bw Day and fare
plt.figure(figsize=(15,7))
plt.scatter(x=train['Day'],y=train['fare_amount'],s=10)
plt.xlabel("Day")
plt.ylabel("Fare")
plt.show()

the highest fare seems to be on sunday,monday and thursday and  low on wednesday and thursday , may be due to low demand on cabs on saturday the cab fare is too low and high demand of cabs on sunday and monday show the high fare price.......

In [ ]:
# relationship bw distance and fare
plt.figure(figsize=(15,7))
plt.scatter(x=train['distance'],y=train['fare_amount'],c='g')
plt.xlabel("distance")
plt.ylabel("Fare")
plt.show()

distance is direcly propotional to fare charge as the distance is increasing fare charge also increasing

# Feacture Scaling

In [ ]:
# check the traning data which is uniformally distributed or not
for i in ['fare_amount','distance']:
    print(i)
    sns.displot(train[i],bins='auto',color='green')
    plt.title("distribution of variable "+i)
    plt.ylabel('density')
    plt.show()
    
    
# right skewed

# log transformation

In [ ]:
# since the fair amt is hihlt skewed applying log transformation
train['fare_amount'] = np.log1p(train['fare_amount'])

# since variable distance is highly skewed
train['distance'] = np.log1p(train['distance'])

In [ ]:
for i in ['fare_amount','distance']:
    print(i)
    sns.displot(train[i],bins='auto',color='green')
    plt.title("distribution of variable "+i)
    plt.ylabel('density')
    plt.show()

In [ ]:
# check the test data which is uniformally distributed or not
for i in ['distance']:
    print(i)
    sns.displot(test[i],bins='auto',color='green')
    plt.title("distribution of variable "+i)
    plt.ylabel('density')
    plt.show()
    

In [ ]:
# since variable distance is highly skewed
test['distance'] = np.log1p(test['distance'])

In [ ]:
# check the test data which is uniformally distributed or not
for i in ['distance']:
    print(i)
    sns.displot(test[i],bins='auto',color='green')
    plt.title("distribution of variable "+i)
    plt.ylabel('density')
    plt.show()
    

# Applying ML  Algo

1 data modelling
2 decision tree
3 random forest
4 gradient boosting
5 Hyper parameter turing

In [ ]:
# train test spilt for further modelling                                          train                 
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,train.columns !='fare_amount'],train.iloc[:,0],test_size=.20,random_state=1)

In [ ]:
X_train

In [ ]:
y_train   #OUTPUT

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_test.shape)

# Linear Regression

In [ ]:
# Building model on top of traning dataset
fit_LR = LinearRegression().fit(X_train,y_train)

In [ ]:
#predicting on traning dataset
pred_train_LR = fit_LR.predict(X_train)

In [ ]:
#predicting on test data
pred_test_LR = fit_LR.predict(X_test)

In [ ]:
#calculating rmse test data
RMSE_test_LR = np.sqrt(mean_squared_error(y_test,pred_test_LR))

# calculating rmse for train data
RMSE_train_LR = np.sqrt(mean_squared_error(y_train,pred_train_LR))

In [ ]:
print("Root mean square error for Traning data ="+str(RMSE_train_LR))
print("Root mean square error for Test data ="+str(RMSE_test_LR))

In [ ]:
# calculate r62 for train data
from sklearn.metrics import r2_score
r2_score(y_train , pred_train_LR)

In [ ]:
r2_score(y_test,pred_test_LR)

# Decision Tree Model

In [ ]:
fit_DT = DecisionTreeRegressor(max_depth=2).fit(X_train,y_train)

In [ ]:
# prediction on train data
pred_train_DT = fit_DT.predict(X_train)

#prediction on test data
pred_test_DT = fit_DT.predict(X_test)

In [ ]:
#calculating rmse test data
RMSE_test_DT = np.sqrt(mean_squared_error(y_test,pred_test_DT))

# calculating rmse for train data
RMSE_train_DT = np.sqrt(mean_squared_error(y_train,pred_train_DT))

In [ ]:
print("Root mean square error for Traning data ="+str(RMSE_train_DT))
print("Root mean square error for Test data ="+str(RMSE_test_DT))

In [ ]:
# calculate r62 for train data
from sklearn.metrics import r2_score
r2_score(y_train , pred_train_DT)

In [ ]:
r2_score(y_test , pred_test_DT)

# Random Forest 

In [ ]:
fit_RF = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

In [ ]:
# prediction on train data
pred_train_RF = fit_RF.predict(X_train)

#prediction on test data
pred_test_RF = fit_RF.predict(X_test)

In [ ]:
#calculating rmse test data
RMSE_test_RF = np.sqrt(mean_squared_error(y_test,pred_test_RF))

# calculating rmse for train data
RMSE_train_RF = np.sqrt(mean_squared_error(y_train,pred_train_RF))

In [ ]:
print("Root mean square error for Traning data ="+str(RMSE_train_RF))
print("Root mean square error for Test data ="+str(RMSE_test_RF))

In [ ]:
# calculate r62 for train data
from sklearn.metrics import r2_score
r2_score(y_train , pred_train_RF)

In [ ]:
r2_score(y_test , pred_test_RF)

# Gradient Boosting

In [ ]:
fit_GB = GradientBoostingRegressor().fit(X_train,y_train)

In [ ]:
# prediction on train data
pred_train_GB = fit_GB.predict(X_train)

#prediction on test data
pred_test_GB = fit_GB.predict(X_test)

In [ ]:
#calculating rmse test data
RMSE_test_GB = np.sqrt(mean_squared_error(y_test,pred_test_GB))

# calculating rmse for train data
RMSE_train_GB = np.sqrt(mean_squared_error(y_train,pred_train_GB))

In [ ]:
print("Root mean square error for Traning data ="+str(RMSE_train_GB))
print("Root mean square error for Test data ="+str(RMSE_test_GB))

In [ ]:
# calculate r62 for train data
from sklearn.metrics import r2_score
r2_score(y_train , pred_train_GB)

In [ ]:
r2_score(y_test , pred_test_GB)

## Parameter turing

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
from pprint import pprint
#look at the parameter used by this
print('parameter currently in used')
pprint(rf.get_params())

In [ ]:
# Random hyperparameter grid
from sklearn.model_selection import train_test_split,RandomizedSearchCV

In [ ]:
## Random search cv on random forest model

RRF = RandomForestRegressor(random_state =0)
n_estimator = list(range(1,20,2))
depth = list(range(1,100,2))

# Create the random grid
rand_grid = {'n_estimators':n_estimator,
              'max_depth' : depth}

randomcv_rf = RandomizedSearchCV(RRF,param_distributions = rand_grid ,n_iter = 5,cv=5,random_state=0)
randomcv_rf = randomcv_rf.fit(X_train,y_train)
predictions_RRF = randomcv_rf.predict(X_test)

view_best_params_RRF = randomcv_rf.best_params_

best_model = randomcv_rf.best_estimator_

predictions_RRF = best_model.predict(X_test)

RRF_r2 = r2_score(y_test,predictions_RRF)

# calculating rmse
RRF_rmse = np.sqrt(mean_squared_error(y_test,predictions_RRF))

print("Random Search cv on Random forest Regressor Model performance: ")
print("Best Parameter:",view_best_params_RRF)
print("R-Squared = {:0.2}".format(RRF_r2))
print("RMSE ",RRF_rmse)

In [ ]:
#Applying random search cv Gradient Boosting
gb = GradientBoostingRegressor(random_state=42)
from pprint import pprint
#look at the parameter used by this
print('parameter currently in used')
pprint(gb.get_params())

In [ ]:
## Random search cv on Gradient Boosting model

gb = GradientBoostingRegressor(random_state=42)
n_estimator = list(range(1,20,2))
depth = list(range(1,100,2))

# Create the random grid
rand_grid = {'n_estimators':n_estimator,
              'max_depth' : depth}

randomcv_gb = RandomizedSearchCV(gb,param_distributions = rand_grid ,n_iter = 5,cv=5,random_state=0)
randomcv_gb = randomcv_gb.fit(X_train,y_train)
predictions_gb = randomcv_gb.predict(X_test)

view_best_params_gb = randomcv_gb.best_params_

best_model = randomcv_gb.best_estimator_

predictions_gb = best_model.predict(X_test)

gb_r2 = r2_score(y_test,predictions_RRF)

# calculating rmse
gb_rmse = np.sqrt(mean_squared_error(y_test,predictions_RRF))

print("Random Search cv Gradient Boosting Model performance: ")
print("Best Parameter:",view_best_params_gb)
print("R-Squared = {:0.2}".format(gb_r2))
print("RMSE ",gb_rmse)

## Grid Search


In [ ]:
## Grid Search CV on random forest model

GRF = RandomForestRegressor(random_state =0)
n_estimator = list(range(11,20,1))
depth = list(range(5,15,12))

# Create the random grid
grid_search = {'n_estimators':n_estimator,
              'max_depth' : depth}

# Grid Search cross validation with 5 folds cv
gridcv_rf = GridSearchCV(RRF,param_grid = grid_search ,cv=5)
gridcv_RF = gridcv_rf.fit(X_train,y_train)
predictions_GRF = gridcv_RF.predict(X_test)

#view_best_params_RRF = gridcv_RF.best_params_

best_model = gridcv_RF.best_estimator_

predictions_GRF = best_model.predict(X_test)

GRF_r2 = r2_score(y_test,predictions_RRF)

# calculating rmse
GRF_rmse = np.sqrt(mean_squared_error(y_test,predictions_GRF))

print("Random Search cv on Random forest Regressor Model performance: ")
#print("Best Parameter:",view_best_params_GRF)
print("R-Squared = {:0.2}".format(GRF_r2))
print("RMSE ",GRF_rmse)

In [ ]:
## Grid Search CV on Gradient Boosting

GGB = GradientBoostingRegressor(random_state=42)
n_estimator = list(range(11,20,1))
depth = list(range(5,15,12))

# Create the random grid
grid_search = {'n_estimators':n_estimator,
              'max_depth' : depth}

# Grid Search cross validation with 5 folds cv
gridcv_GB = GridSearchCV(GGB,param_grid = grid_search ,cv=5)
gridcv_GB = gridcv_rf.fit(X_train,y_train)
predictions_GGB = gridcv_GB.predict(X_test)

view_best_params_GGB = gridcv_GB.best_params_

best_model = gridcv_GB.best_estimator_

predictions_GGB = best_model.predict(X_test)

GGB_r2 = r2_score(y_test,predictions_RRF)

# calculating rmse
GGB_rmse = np.sqrt(mean_squared_error(y_test,predictions_GGB))

print("Grid Search CV on Gradient Boosting Model performance: ")
print("Best Parameter:",view_best_params_GGB)
print("R-Squared = {:0.2}".format(GGB_r2))
print("RMSE ",GGB_rmse)

## Predictions of fare from provided test  dataset

In [ ]:
## Grid Search CV for random forest model

regr = RandomForestRegressor(random_state =0)
n_estimator = list(range(11,20,1))
depth = list(range(5,15,2))

# Create the random grid
grid_search = {'n_estimators':n_estimator,
              'max_depth' : depth}

# Grid Search cross validation with 5 folds cv
gridcv_rf = GridSearchCV(regr,param_grid = grid_search ,cv=5)
gridcv_rf = gridcv_rf.fit(X_train,y_train)
predictions_GRF = gridcv_rf.predict(X_test)

view_best_params_RRF = gridcv_RF.best_params_

# Apply model on Test data
predictions_GRF_test = gridcv_rf.predict(test)




In [ ]:
predictions_GRF_test

In [ ]:
test['predicted_fare'] = predictions_GRF_test

In [ ]:
test.head()

In [ ]:
test.tail()

##  OUTPUT  STORING IN FILE 

In [ ]:
test.to_csv("test.csv")